In [20]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect("C:\\Users\\lalan\\OneDrive\\Desktop\\IMDB")

filter_df = pd.read_sql_query("SELECT * FROM filtered_titles", cnx)

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Set the 'imdbID' column as the index
# df.set_index('id', inplace=True)

# Concatenate 'Plot' and 'Genre' columns
df['Plot_Genre'] = df['ptitle'] + ' ' + df['genres'] + ' ' + df['directors']  + ' ' + df['writers']

# Define a TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(df['Plot_Genre'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(imdb_id, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the IMDb ID
    idx = df.index.get_loc(imdb_id)

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df.iloc[movie_indices]



In [23]:
# Example: Get recommendations for a movie by IMDb ID
movie_imdb_id = "tt0120338"  # IMDb ID for "The Godfather"
recommendations = get_recommendations(movie_imdb_id)
print("Recommendations for the movie with IMDb ID", movie_imdb_id, ":")
recommendations


Recommendations for the movie with IMDb ID tt0120338 :


,type,ptitle,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,Plot_Genre
id,,,,,,,,,,
tt0096754,movie,The Abyss,1989,140,"Adventure,Drama,Mystery",7.5,194810,nm0000116,nm0000116,"The Abyss Adventure,Drama,Mystery nm0000116 nm..."
tt0499549,movie,Avatar,2009,162,"Action,Adventure,Fantasy",7.9,1392479,nm0000116,nm0000116,"Avatar Action,Adventure,Fantasy nm0000116 nm00..."
tt0088247,movie,The Terminator,1984,107,"Action,Adventure,Sci-Fi",8.1,929411,nm0000116,"nm0000116,nm0005036,nm0936537","The Terminator Action,Adventure,Sci-Fi nm00001..."
tt0103064,movie,Terminator 2: Judgment Day,1991,137,"Action,Adventure,Sci-Fi",8.6,1180572,nm0000116,"nm0000116,nm0936537","Terminator 2: Judgment Day Action,Adventure,Sc..."
tt0090605,movie,Aliens,1986,137,"Action,Adventure,Sci-Fi",8.4,767288,nm0000116,"nm0000116,nm0318429,nm0001353,nm0639321,nm0795953","Aliens Action,Adventure,Sci-Fi nm0000116 nm000..."
tt0111503,movie,True Lies,1994,141,"Action,Comedy,Thriller",7.3,282516,nm0000116,"nm0000116,nm0956022,nm0584770,nm0436687","True Lies Action,Comedy,Thriller nm0000116 nm0..."
tt1630029,movie,Avatar: The Way of Water,2022,192,"Action,Adventure,Fantasy",7.5,500019,nm0000116,"nm0000116,nm0415425,nm0798646,nm0295264,nm0004307","Avatar: The Way of Water Action,Adventure,Fant..."
tt0297144,movie,Ghosts of the Abyss,2003,61,"Documentary,Family",6.8,6422,nm0000116,\N,"Ghosts of the Abyss Documentary,Family nm00001..."
tt0417415,movie,Aliens of the Deep,2005,100,"Documentary,Family",6.3,3439,"nm0000116,nm0702797",\N,"Aliens of the Deep Documentary,Family nm000011..."


In [31]:
import sqlite3
import pandas as pd

# Assuming 'cnx' is a SQLAlchemy engine or a connection
cnx = sqlite3.connect('your_database.db')

# Read the CSV file into a DataFrame
fdf = pd.read_csv('filtered.csv')

# Drop the '_id' column
fdf.drop('_id', axis=1, inplace=True)

# Set 'imdbID' as the index
fdf.set_index('imdbID', inplace=True)

# Define the table name
table_name = 'filtered'

# Check if the table already exists
cursor = cnx.cursor()
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
existing_table = cursor.fetchone()

# If the table doesn't exist, create it
if not existing_table:
    fdf.to_sql(table_name, cnx)
    print(f"Table '{table_name}' created successfully.")
else:
    print(f"Table '{table_name}' already exists.")

# Commit changes and close the connection
cnx.commit()
cnx.close()


Table 'filtered' created successfully.
